In [2]:
from selenium import webdriver
import lxml
import time
import requests
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd

options = webdriver.ChromeOptions()

options.add_argument('headless')

options.add_argument('window-size=1920x1080')

path = 'chromedriver.exe'

In [5]:
driver = webdriver.Chrome(path, options = options)
driver.implicitly_wait(3)
driver.get("http://www.kobis.or.kr/kobis/business/stat/boxs/findDailyBoxOfficeList.do")
driver.implicitly_wait(4)
num_list = ['0','104','208','308','420','537','643'] #버튼들의 번호들을 찾아서 저장해둔다

for i in num_list:
    #각각 버튼들을 네번씩 눌러 50위까지 펼쳐주기
    for j in range(4):

        driver.find_element_by_id("btn_{}".format(i)).click()

        driver.implicitly_wait(2)


daily_boxoffice_html = driver.page_source

parsed_daily_boxoffice_html = bs(daily_boxoffice_html, 'html.parser')

boxoffice_list = []
for i in range(7):
    #string formatting을 사용해서 데이터플레임 한번에 다 만들어주기
    daily_boxoffice = parsed_daily_boxoffice_html.find("tbody", {"id":"tbody_{}".format(num_list[i])})

    movie_list = daily_boxoffice.find_all("tr")

    daily_movies = {'영화명' : [],
    '순위 변동' : [],
    '개봉일' : [],
    '매출액' : [],
    '매출액점유율' : [],
    '관객수' : [],
    '누적관객수' : [],
    '스크린수' : [],
    '상영횟수' : []}

    for movie in movie_list:
        title_rank = movie.find("td").find_next_sibling("td").text
        daily_movies['영화명'].append(title_rank[:-5].strip())
        daily_movies['순위 변동'].append(title_rank[-4:].strip())
        daily_movies['개봉일'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['매출액'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['매출액점유율'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['누적관객수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['스크린수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())
        daily_movies['상영횟수'].append(movie.find("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").find_next_sibling("td").text.strip())

    boxoffice_list.append(pd.DataFrame(daily_movies)) 

In [6]:
import sqlite3

con = sqlite3.connect("daily_boxoffice.db")
date = 11
for i in range(7):
    con.execute("CREATE TABLE IF NOT EXISTS boxofficeDec{}(title text, rank int, open text, earning int, earning_share text, audiences int, total_audiences int, num_screen int, num_play int)".format(str(date)))
    boxoffice_list[i].to_sql("boxofficeDec{}".format(str(date)), con, chunksize=1000, if_exists='replace')
    date -= 1

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [7]:
con.commit()

con.close()

In [18]:
#확인
con = sqlite3.connect("daily_boxoffice.db")

cursor = con.cursor()

cursor.execute("SELECT * FROM boxofficeDec5")

data = cursor.fetchall()

df = pd.DataFrame(data)

con.close()

display(df)

,0,1,2,3,4,5,6,7,8,9
0,0,조커,동일,2019-10-02,"4,603,149,150",53.2%,"514,323","1,784,983","1,418","7,214"
1,1,가장 보통의 연애,동일,2019-10-02,"2,160,079,360",25.0%,"247,284","847,505","1,058","5,303"
2,2,퍼펙트맨,동일,2019-10-02,"1,092,868,670",12.6%,"126,183","548,464",803,"3,142"
3,3,장사리 : 잊혀진 영웅들,동일,2019-09-25,"322,713,070",3.7%,"38,363","1,004,884",650,"1,516"
4,4,소피와 드래곤: 마법책의 비밀,동일,2019-10-02,"99,564,920",1.2%,"12,538","58,570",469,705
5,5,양자물리학,1상승,2019-09-25,"69,905,230",0.8%,"8,155","536,469",376,610
6,6,몬스터 하우스,1하락,2019-10-03,"61,215,900",0.7%,"7,434","31,341",202,312
7,7,나쁜 녀석들: 더 무비,동일,2019-09-11,"52,075,890",0.6%,"5,941","4,552,253",271,434
8,8,원스 어폰 어 타임... 인 할리우드,1상승,2019-09-25,"47,697,780",0.6%,"5,307","253,986",147,220
9,9,47미터 2,1상승,2019-08-28,"2,637,000",0.0%,"2,407","489,723",7,22
